# Esame Artificial Intelligence

## Informazioni

Cognome: **Stenner Talini** <br>
Nome: **Davide** <br>
Data: 09/06/2019

## Testo Esame

Quesito 1 - Problemi di soddisfacimento dei vincoli <br>

Rappresentare come CSP il seguente problema e risolverlo tramite backtracking_search e AC-3 (sia
combinati, con l’opzione inference=mac di backtracking_search, sia usati indipendentemente) del
notebook csp.ipynb:

**Un quadrato magico consiste nell’inserire in una matrice quadrata di lato N > 1, gli N2 numeri interi
compresi tra 1 e N2, facendo in modo che, sommando i numeri presenti in ogni riga, in ogni colonna e
in entrambe le diagonali principali, si ottenga sempre lo stesso valore (detto costante magica).
Considerare solo il caso N = 3 e la costante magica 15 (ottenuta considerando che la somma dei nove
numeri da inserire, da 1 a 9, è uguale a 45 e che il quadrato contiene tre righe e che quindi il valore
della costante magica deve essere necessariamente uguale a: 45/3 = 15).**


In [167]:
from csp import *
from notebook import psource, pseudocode, plot_NQueens
import numpy as np
import pandas as pd
import copy
from platform import python_version
%matplotlib inline

# Hide warnings in the matplotlib sections
import warnings
warnings.filterwarnings("ignore")

print('Nota:\nVersioni utilizzate:','\nNumpy: ',np.version.version,'\nPandas: ',pd.__version__,'\nPython: ',python_version())

Nota:
Versioni utilizzate: 
Numpy:  1.16.4 
Pandas:  0.24.2 
Python:  3.7.3


## Definizione Problema

### Variabili

Per risolvere il problema del cubo magico ho definito due tipologie di vincoli che hanno condizionato la definizione problema:
* Ogni elemento del cubo deve soddisfare: \begin{equation*} X_{i}\neq X_{k}   \forall {i,k} \end{equation*} Ossia tutti gli elementi del cubo devono differire tra loro, in modo tale da associare tutti i numeri da {1,...,9} <br>
* Ogni riga,colonna e diagonale del cubo deve dare come somma dei propri elementi il numero magico, nel nostro caso 15.

Dato che nei problemi csp i vincoli vanno rappresentati in forma binaria bisogna introdurre delle variabili fittizie che possano immettere il vincolo sulle tre variabili, ossia che abbiano somma pari a 15.

Le variabili quindi possono essere divise in due categorie: <br>
* Variabili Cella: Le singole caselle del cubo magico : \begin{equation*} X_{0},...,X_{8} \end{equation*} <br> Ordinate da sinistra a destra e dall'alto in basso in modo tale che il cubo magico nella sua forma originali sia: <br>


In [154]:
pd.DataFrame([['X0','X1','X2'],['X3','X4','X5'],['X6','X7','X8']])


,0,1,2
0,X0,X1,X2
1,X3,X4,X5
2,X6,X7,X8


* Variabili fittizie: variabili create per codificare i vincoli sulle somme pari a 15 e trasformare i vincoli da vincoli a 3 variabili a vincoli binari (vincoli necessari per la formulazione del problema csp). Queste variabili fittizie sono state chiamate R0, R1, R2, C0, C1, C2, D0, D1 ossia rappresentano le righe, colonne e diagonali su cui si devono imporre i vincoli. <br>Perciò per esempio *R0 = [X0,X1,X2]* è la riga 1 del cubo e *D0=[X0,X4,X8]* è la diagonale principale. <br> <br>

### Neighbors   

Dopo aver definito le variabili da utilizzare: X0, X1, X2, X3, X4, X5, X6, X7, X8 R0, R1, R2, C0, C1, C2, D0, D1, devo definire i neighbors da passare alla funzione CSP ossia devo definire le connessioni tra le diverse variabili in base ai vincoli da imporre.<br><br> Le variabili riferite alle celle (definite nell'implementazione come 0,1,2,...,8) devono essere collegate a tutte le altre celle del cubo, per soddisfare il vincolo 1 ossia che tutti gli elementi siano diverse tra loro, in modo tale da assegnare tutti i numeri da {1,...,9}, inoltre devono anche essere collegate con le righe, colonne e diagonali (R0,R1,...) di appartenenza per un vincolo che sarà spiegato a seguire.<br><br> Le variabili fittizie (R0,...,D1) devono essere collegate con le variabili riferite alle celle (X0,...,X8) che rappresentano, perciò la variabile R0=[X0,X1,X2] dovrà essere collegata con X0,X1,X2. Inoltre le variabili fittizie dovranno avere un collegamento con le variabili fittizie con cui hanno un elemento in comune. <br>Esempio: R0=[X0,X1,X2] e C0=[X0,X3,X6] hanno X0 in comune perciò vanno collegate tra loro. Il motivo di questa definizione del problema verrà spiegata nella sezione Vincoli.<br> Per concludere il dizionario neighbors è così definito:

In [155]:
Dim=3
Magic=Dim*(Dim**2+1)/2
Matrix=np.array(range(0,Dim**2)).reshape((Dim,Dim))
neighboursAll={}
Neighbours={}
for i in range(0,Dim):
    for k in range(0,Dim):
        neighboursAll[Matrix[i,k]]=np.delete(np.arange(Dim**2),Matrix[i,k]).tolist()+['R'+str(i),'C'+str(k)]
        if i==k:
            neighboursAll[Matrix[i,k]]=neighboursAll[Matrix[i,k]]+['D0']
        if i+k==Dim-1:
            neighboursAll[Matrix[i,k]]=neighboursAll[Matrix[i,k]]+['D1']
#RIGHE
for i in range(Dim):
    neighboursAll['R'+str(i)]=Matrix[i,:].tolist()+['C'+str(k) for k in range(Dim) ] + ['D'+str(k) for k in range(2)]
    Neighbours['R'+str(i)]=Matrix[i,:].tolist()
#COLONNE
for i in range(Dim):
    neighboursAll['C'+str(i)]=Matrix[:,i].tolist()+['R'+str(k) for k in range(Dim) ] + ['D'+str(k) for k in range(2) ]
    Neighbours['C'+str(i)]=Matrix[:,i].tolist()
#DIAGONALE
for i in range(2):
    neighboursAll['D'+str(i)]=[Matrix.diagonal(),np.fliplr(Matrix).diagonal()][i].tolist()+['R'+str(k) for k in range(Dim) ] + ['C'+str(k) for k in range(Dim) ] 
    if Dim%2!=0:
        neighboursAll['D'+str(i)]=neighboursAll['D'+str(i)]+['D'+str(1-i)]    
    Neighbours['D'+str(i)]=[Matrix.diagonal(),np.fliplr(Matrix).diagonal()][i].tolist()
print('Dizionario Neighbors:\n')
neighboursAll

Dizionario Neighbors:



{0: [1, 2, 3, 4, 5, 6, 7, 8, 'R0', 'C0', 'D0'],
 1: [0, 2, 3, 4, 5, 6, 7, 8, 'R0', 'C1'],
 2: [0, 1, 3, 4, 5, 6, 7, 8, 'R0', 'C2', 'D1'],
 3: [0, 1, 2, 4, 5, 6, 7, 8, 'R1', 'C0'],
 4: [0, 1, 2, 3, 5, 6, 7, 8, 'R1', 'C1', 'D0', 'D1'],
 5: [0, 1, 2, 3, 4, 6, 7, 8, 'R1', 'C2'],
 6: [0, 1, 2, 3, 4, 5, 7, 8, 'R2', 'C0', 'D1'],
 7: [0, 1, 2, 3, 4, 5, 6, 8, 'R2', 'C1'],
 8: [0, 1, 2, 3, 4, 5, 6, 7, 'R2', 'C2', 'D0'],
 'R0': [0, 1, 2, 'C0', 'C1', 'C2', 'D0', 'D1'],
 'R1': [3, 4, 5, 'C0', 'C1', 'C2', 'D0', 'D1'],
 'R2': [6, 7, 8, 'C0', 'C1', 'C2', 'D0', 'D1'],
 'C0': [0, 3, 6, 'R0', 'R1', 'R2', 'D0', 'D1'],
 'C1': [1, 4, 7, 'R0', 'R1', 'R2', 'D0', 'D1'],
 'C2': [2, 5, 8, 'R0', 'R1', 'R2', 'D0', 'D1'],
 'D0': [0, 4, 8, 'R0', 'R1', 'R2', 'C0', 'C1', 'C2', 'D1'],
 'D1': [2, 4, 6, 'R0', 'R1', 'R2', 'C0', 'C1', 'C2', 'D0']}

### Dominio

Il dominio delle variabili si scompone in due categorie:<br>

* Variabili Celle: Il dominio delle variabili è \begin{equation*} D_{X_{i}} \in \{1,...,9\} \forall i\end{equation*} <br>
* Variabili Fittizie: Il dominio delle variabili fittizie è rappresentato da tutte le combinazioni possibili di numeri da 1 a 9 nelle tre celle che diano come somma 15 ed incorporano quindi i vincoli iniziali sulle somme sulle righe,colonne e diagonali<br>

Facendo tutte le possibili combinazioni di 3 numeri che diano come somma 15 si ottiene:

In [156]:
def expand_grid(data_dict):
  rows = itertools.product(*data_dict.values())
  return pd.DataFrame.from_records(rows, columns=data_dict.keys())
Vincoli={}
for i in range(Dim):
    Vincoli[i]=np.arange(1,Dim**2+1)
Vincoli=expand_grid(Vincoli)
Vincoli=Vincoli[Vincoli.sum(axis=1)==Magic]
print('Numero delle Combinazioni Totali:  ',Vincoli.shape[0])
print('Esempio delle variabili fittizie: ')
Vincoli.head()


Numero delle Combinazioni Totali:   61
Esempio delle variabili fittizie: 


,0,1,2
44,1,5,9
52,1,6,8
60,1,7,7
68,1,8,6
76,1,9,5


Ogni Variabile fittizia ha un dominio con cardinalità 61. Questo dominio può essere semplificato, infatti dato che nel cubo magico nessun valore si può ripetere, è lecito imporre che siano scartate tutte quelle combinazioni in cui ci siano dei valori ripetuti, per esempio [7,4,4],...
Da ciò si ha che il dominio delle variabili fittizie si riduce a 48 unità.

In [157]:
ToErase=[]
for i in range(Vincoli.shape[0]):
    if len(Vincoli.iloc[i,:].tolist()) != len(set(Vincoli.iloc[i,:].tolist())):
        ToErase=ToErase+[Vincoli.index.values[i]]
Vincoli=Vincoli.drop(ToErase,axis=0)
print('Numero delle Combinazioni Totali:  ',Vincoli.shape[0])
print('Numero di Combinazioni Tolte:  ',len(ToErase))


Numero delle Combinazioni Totali:   48
Numero di Combinazioni Tolte:   13


Il dizionario dei diversi domini è così definito:

In [158]:
Dom_Vincoli=[]

for row in Vincoli.iterrows():
    index, data = row
    Dom_Vincoli.append(data.tolist())

domains={}
#Domini variabili Cella
for i in range(Dim**2):
    domains[list(neighboursAll.keys())[i]]=np.arange(1,Dim**2+1).tolist()
#Domini variabili Fittizie
for i in range(Dim**2,len(neighboursAll)):
    domains[list(neighboursAll.keys())[i]]=Dom_Vincoli
domains

{0: [1, 2, 3, 4, 5, 6, 7, 8, 9],
 1: [1, 2, 3, 4, 5, 6, 7, 8, 9],
 2: [1, 2, 3, 4, 5, 6, 7, 8, 9],
 3: [1, 2, 3, 4, 5, 6, 7, 8, 9],
 4: [1, 2, 3, 4, 5, 6, 7, 8, 9],
 5: [1, 2, 3, 4, 5, 6, 7, 8, 9],
 6: [1, 2, 3, 4, 5, 6, 7, 8, 9],
 7: [1, 2, 3, 4, 5, 6, 7, 8, 9],
 8: [1, 2, 3, 4, 5, 6, 7, 8, 9],
 'R0': [[1, 5, 9],
  [1, 6, 8],
  [1, 8, 6],
  [1, 9, 5],
  [2, 4, 9],
  [2, 5, 8],
  [2, 6, 7],
  [2, 7, 6],
  [2, 8, 5],
  [2, 9, 4],
  [3, 4, 8],
  [3, 5, 7],
  [3, 7, 5],
  [3, 8, 4],
  [4, 2, 9],
  [4, 3, 8],
  [4, 5, 6],
  [4, 6, 5],
  [4, 8, 3],
  [4, 9, 2],
  [5, 1, 9],
  [5, 2, 8],
  [5, 3, 7],
  [5, 4, 6],
  [5, 6, 4],
  [5, 7, 3],
  [5, 8, 2],
  [5, 9, 1],
  [6, 1, 8],
  [6, 2, 7],
  [6, 4, 5],
  [6, 5, 4],
  [6, 7, 2],
  [6, 8, 1],
  [7, 2, 6],
  [7, 3, 5],
  [7, 5, 3],
  [7, 6, 2],
  [8, 1, 6],
  [8, 2, 5],
  [8, 3, 4],
  [8, 4, 3],
  [8, 5, 2],
  [8, 6, 1],
  [9, 1, 5],
  [9, 2, 4],
  [9, 4, 2],
  [9, 5, 1]],
 'R1': [[1, 5, 9],
  [1, 6, 8],
  [1, 8, 6],
  [1, 9, 5],
  [2, 4, 9],
 

### Vincoli

Ci sono 4 tipologie di vincoli binari in base alle variabili A,B (con i relativi valori a,b) scelte:<br>
1. Se A e B sono Variabili Cella allora: \begin{equation*} a \neq b \end{equation*} <br>
2. Se A è una variabile Cella e B è una variabile fittizia (o viceversa):<br> Sia il valore k appartente a b( che è un vettore rappresentante una riga,colonna o diagonale), il valore della cella di B che rappresenta la cella A allora : \begin{equation*} a = k \end{equation*} <br> Ossia le variabili fittizie che fanno riferimento alle variabili cella devono avere lo stesso valore nelle celle corrispondenti.<br>Esempio : se A=X0, B=R0=[X0,X1,X2], a il valore di A e b=[b0,b1,b2] i valori riferiti a [X0,X1,X2]. Allora il valore b0 riferito ad X0 dovrà essere uguale ad a.<br><br>
3. Se A e B sono variabili Fittizie allora: i valori delle celle che hanno in comune A e B devono essere uguali.<br> Esempio: sia A=R0=[X0,X1,X2], B=C0=[X0,X3,X6], a=[a0,a1,a2] e b=[b0,b3,b6] allora a0 dovrà essere uguale a b0.<br>

Il vincolo 1 serve a fare in modo che tutti i valori siano scelti e che tutte le celle siano diverse tra loro.<br>I vincoli 2 - 3 servono a mantenere coerenza tra le variabili Fittizie (che hanno per costruzione il vincolo : somma = 15) ed anche con le varibili Cella

In [181]:
#DEFINIZIONE VINCOLI
def Sum_Constraint(A,a,B,b):
    if type(A)!=str and type(B)!=str: #vincolo1
        return(a!=b)
    if type(A)==str and type(B)!=str:
        for index, s in enumerate(Neighbours[A]): #vincolo2
            if s==B:
                break
        return(a[index]==b)
    if type(B)==str and type(A)!=str:
        for index, s in enumerate(Neighbours[B]): #vincolo2
            if s==A:
                break
        return(b[index]==a)
    if type(B)==str and type(A)==str: #vincolo3
        both=list(set(Neighbours[A]).intersection(Neighbours[B]))
        indices_A = [Neighbours[A].index(x) for x in both][0]
        indices_B = [Neighbours[B].index(x) for x in both][0]
        return(a[indices_A]==b[indices_B])
    
#DEFINISCO IL CSP
def MagicCPS(domains, neighbors):  
    if isinstance(neighbors, str):
        neighbors = parse_neighbors(neighbors)
    return make_instru(CSP(list(neighbors.keys()), domains, neighbors,
               Sum_Constraint))
#TEST GOAL
def Magic_Test(R):
    if (sum(R.sum(axis=0)==Magic)+sum(R.sum(axis=1)==Magic)+
        np.array(sum(R.diagonal())==Magic) + np.array(sum(np.fliplr(R).diagonal())==Magic))==(2*Dim+2):
        return(True)
    else:
        return(False)
#RICAVA SOLUZIONE DAL RISULTATO
def CalcSol(result,M,Dim=Dim):
    Result=M.copy()
    for i in range(0,Dim):
        for k in range(0,Dim):
            Result[i,k]=result[M[i,k]]
    return(Result)
#DISPLAY INFORMAZIONI
def displaySolInfo(R,Sol):
    Soluzione=Magic_Test(R)
    Assign=len(Sol.assignment_history)
    print('La soluzione trovata è corretta, ossia da come somma (',Magic,') su tutte le righe, colonne e diagonali?  :', Soluzione,'\n')
    print('La lunghezza della storicità degli assegnamenti è: ',Assign,'\n')
    print('La soluzione è :\n\n\n',R)

# Modifica del CSP per definire assignment_history
class InstruCSP(CSP):
    
    def __init__(self, variables, domains, neighbors, constraints):
        super().__init__(variables, domains, neighbors, constraints)
        self.assignment_history = []
        
    def assign(self, var, val, assignment):
        super().assign(var,val, assignment)
        self.assignment_history.append(copy.deepcopy(assignment))
    
    def unassign(self, var, assignment):
        super().unassign(var,assignment)
        self.assignment_history.append(copy.deepcopy(assignment))
        
def make_instru(csp):
    return InstruCSP(csp.variables, csp.domains, csp.neighbors, csp.constraints)

## Risoluzione Problema

Avendo definito le variabili, i domini, i vincoli ed i collegamenti passo a risolvere utilizzando l'algoritmo backtracking_search.

In [182]:
Sol=MagicCPS(domains, neighboursAll) #definisco il CSP
result = backtracking_search(Sol) #applico backtracking_search
Result=CalcSol(result=result,M=Matrix) #ricavo la matrice del risultato finale
displaySolInfo(Result,Sol) # display del soddisfacimento del test , del numero di iterazioni e della soluzione.

La soluzione trovata è corretta, ossia da come somma ( 15.0 ) su tutte le righe, colonne e diagonali?  : True 

La lunghezza della storicità degli assegnamenti è:  236813 

La soluzione è :


 [[2 9 4]
 [7 5 3]
 [6 1 8]]


Prima di applicare backtracking_search e AC3, provo ad applicare solo AC3 per testare la consistenza d'arco.

In [183]:
Sol=MagicCPS(domains, neighboursAll)
removals = []
Cons_Arc=AC3(Sol, removals=removals)
print('La configurazione è consistente?  :',Cons_Arc)

La configurazione è consistente?  : True


Dato che la configurazione è consistente applico backtracking_search e AC3.

In [184]:
Sol=MagicCPS(domains, neighboursAll)
result = backtracking_search(Sol,inference=mac)
Result=CalcSol(result=result,M=Matrix)
displaySolInfo(Result,Sol)


La soluzione trovata è corretta, ossia da come somma ( 15.0 ) su tutte le righe, colonne e diagonali?  : True 

La lunghezza della storicità degli assegnamenti è:  19 

La soluzione è :


 [[2 9 4]
 [7 5 3]
 [6 1 8]]


L'utilizzo di AC-3 ha portato ad una notevole riduzione del numero del numero di assegnamenti passando da 236813 a 19. Questo beneficio è dovuto al fatto che tramite la consistenza d'arco i domini delle variabili fittizie per il vincolo 3 si riducono notevolmente lasciando solamente quelle poche combinazioni di numeri (per righe,colonne e diagonali) coerenti tra loro la cui somma fa 15.

Suppongo di mettere il valore vero nella casella 0, ossia 2.

In [190]:
domains2=domains.copy()
domains2[0]=[Result[0][0]]
Sol=MagicCPS(domains2, neighboursAll)
result = backtracking_search(Sol,inference=mac)
Result=CalcSol(result=result,M=Matrix)
displaySolInfo(Result,Sol)

La soluzione trovata è corretta, ossia da come somma ( 15.0 ) su tutte le righe, colonne e diagonali?  : True 

La lunghezza della storicità degli assegnamenti è:  17 

La soluzione è :


 [[2 7 6]
 [9 5 1]
 [4 3 8]]


Come si può vedere il numero di assegnamenti si è ridotto da 19 a 17. Ciò perchè fissando un'elemento di una Varibile cella si fissa il suo dominio, perciò AC-3 eliminerà ancora più elementi dai domini delle altre variabili, elementi che precedentemente non venivano eliminati, dato che il dominio della variabile cella conteneva altre unità.